In [2]:
function clone (t) -- deep-copy a table
    if type(t) ~= "table" then return t end
    local meta = getmetatable(t)
    local target = {}
    for k, v in pairs(t) do
        if type(v) == "table" then
            target[k] = clone(v)
        else
            target[k] = v
        end
    end
    setmetatable(target, meta)
    return target
end

In [6]:
-- some data:
x = torch.randn(1,20)
y = torch.Tensor(1,10):zero() y[1][3] = 1

In [7]:
-- R-op (mine) but this is more complicated than Hv = Grad(g^T v) method. 
-- I'm taking the ddf = grad_r (grad_w(w + rv) ), and takes the r part of ddf
-- I should ditch this and implement the easier one. 

local d = require 'autograd'
local t = require 'torch'
d.optimize(true)

params = {
   W = {
      t.randn(20,15),
      t.randn(15,10),
   },
   b = {
      t.randn(15),
      t.randn(10),
   },
    r = {
            W = {
              t.randn(20,15),
              t.randn(15,10),
               },
            b = {
              t.randn(15),
              t.randn(10),
               }
    },
    v = {
           -- has to be the same vector as in Hv
            W = {
              t.randn(20,15),
              t.randn(15,10),
               },
            b = {
              t.randn(15),
              t.randn(10),
               }
    },
    Wb ={    W = {
      t.randn(20,15),
      t.randn(15,10),
   },
   b = {
      t.randn(15),
      t.randn(10),
   }}
}
-- define model
local innerFn = function(params, x, y)
   local h1 = t.tanh(x * params.W[1] + params.b[1])
   local h2 = t.tanh(h1 * params.W[2] + params.b[2])
   local yHat = h2 - t.log(t.sum(t.exp(h2)))
   local loss = - t.sum(t.cmul(yHat, y))
   return loss
end

--print(#params2.r.W)

--TO DO : change params2.Wb + t.cmul(params2.r, params2.v) so that it handles the nested params nicely. 
local outerFn = function(params)
    local product = clone(params.Wb)
    for i = 1, #params.r.W do -- has to change if I want to do a different param configuration
            product.W[i] = t.cmul(params.r.W[i], params.v.W[i]) 
            product.b[i] = t.cmul(params.r.b[i], params.v.b[i]) 
    end
    local addition = clone(params.Wb)
    for i = 1, #params.r.W do -- has to change if I want to do a different param configuration
            addition.W[i] = t.add(params.W[i], product.W[i])
            addition.b[i] = t.add(params.b[i], product.b[i])
    end   
    local grads, loss = d(innerFn)(addition, x, y)
    return loss
end

local ddf = d(outerFn)
--gradGrads = d(outerFn)(params)


-- local params2 = {
--     W = 5,
--     r = 3,
--     v = 2
-- }

gradGrads = ddf(params)

In [13]:
gradGrads.r.W[1].raw 

Columns 1 to 6
-1.2319e-04  3.4769e-11 -1.9364e-06  4.0355e-06 -1.8190e-03 -1.3384e-09
-4.3352e-05  6.5389e-12 -3.9772e-06  2.2154e-05 -1.7886e-03  1.3819e-10
-2.3108e-04 -3.0768e-11  6.8264e-07  3.3153e-05 -1.6447e-03  1.2800e-09
-1.5329e-03 -2.4090e-11 -1.8689e-05  7.8916e-05  4.2790e-03 -1.7870e-08
-1.0816e-03 -1.1619e-10  2.7880e-06  5.4621e-05  3.1012e-03  1.6503e-09
-4.2721e-04  1.0758e-10 -1.9012e-06 -3.6210e-05  7.2588e-04  1.4795e-09
 8.6955e-04 -2.1163e-10  6.4921e-06  3.9250e-05 -5.6689e-04  3.1359e-10
-9.6196e-04  9.6830e-11  1.0248e-05  6.1108e-05 -4.2449e-03 -4.5249e-09
-7.2045e-04  5.7632e-11 -1.0357e-05 -3.5908e-05  8.3325e-03  6.2060e-09
 7.5839e-04  2.2670e-11 -2.5747e-06 -8.4386e-06 -5.2687e-03 -8.5392e-09
-4.8854e-06 -2.9747e-12 -1.8510e-07 -6.0776e-07  7.4283e-05 -2.0005e-10
-4.1352e-04 -5.0327e-11 -4.4916e-06  6.7271e-06  9.6907e-04 -5.1175e-10
 1.4079e-03 -2.5326e-10  1.4732e-05  5.6994e-06 -2.0705e-02 -3.3537e-09
-2.5517e-03  9.7412e-11 -1.8903e-06 -3.5973e-05 -

In [17]:
local input_size = 2
local hidden_size1 = 3
local output_size = 2
x = torch.randn(1,input_size)
y = torch.Tensor(1,output_size):zero() y[1][2] = 1
print(y)

 0  1
[torch.DoubleTensor of size 1x2]



In [ ]:
local d = require 'autograd'
local t = require 'torch'
d.optimize(true)

local input_size = 2
local hidden_size1 = 3
local output_size = 2

-- some data:
x = torch.randn(1,input_size)
y = torch.Tensor(1,output_size):zero() y[1][2] = 1

params = {
   W = {
      t.randn(input_size,hidden_size1),
      t.randn(hidden_size1,output_size),
   },
   b = {
      t.randn(hidden_size1),
      t.randn(output_size),
   },
    r = {
            W = {
              t.randn(input_size, hidden_size1),
              t.randn(hidden_size1, output_size),
               },
            b = {
              t.randn(hidden_size1),
              t.randn(output_size),
               }
    },
    v = {
           -- has to be the same vector as in Hv
            W = {
              t.randn(input_size, hidden_size1),
              t.randn(hidden_size1, output_size),
               },
            b = {
              t.randn(hidden_size1),
              t.randn(output_size),
               }
    },
    Wb ={    W = {
      t.randn(input_size, hidden_size1),
      t.randn(hidden_size1,output_size),
   },
   b = {
      t.randn(hidden_size1),
      t.randn(output_size),
   }}
}
-- define model
local innerFn = function(params, x, y)
   local h1 = t.tanh(x * params.W[1] + params.b[1])
   local h2 = t.tanh(h1 * params.W[2] + params.b[2])
   local yHat = h2 - t.log(t.sum(t.exp(h2)))
   local loss = - t.sum(t.cmul(yHat, y))
   return loss
end

--print(#params2.r.W)

--TO DO : change params2.Wb + t.cmul(params2.r, params2.v) so that it handles the nested params nicely. 
local outerFn = function(params)
    local product = clone(params.Wb)
    for i = 1, #params.r.W do -- has to change if I want to do a different param configuration
            product.W[i] = t.cmul(params.r.W[i], params.v.W[i]) 
            product.b[i] = t.cmul(params.r.b[i], params.v.b[i]) 
    end
    local addition = clone(params.Wb)
    for i = 1, #params.r.W do -- has to change if I want to do a different param configuration
            addition.W[i] = t.add(params.W[i], product.W[i])
            addition.b[i] = t.add(params.b[i], product.b[i])
    end   
    local grads, loss = d(innerFn)(addition, x, y)
    return loss
end

local ddf = d(outerFn)
--gradGrads = d(outerFn)(params)


-- local params2 = {
--     W = 5,
--     r = 3,
--     v = 2
-- }

gradGrads = ddf(params)